# Proyecto: Clasificación de Productos con Aprendizaje por Refuerzo (Q-Learning)

Este proyecto simula una cinta transportadora como las que se encuentran en fábricas o plantas industriales.  
Un agente inteligente aprende a tomar decisiones por sí mismo sobre:

- Cuándo retirar un producto defectuoso.
- Cuándo dejar pasar un producto en buen estado.

Para esto se utiliza **Aprendizaje por Refuerzo**, específicamente el algoritmo **Q-Learning**, donde el agente aprende mediante recompensas.


In [1]:
import numpy as np
import random
import time
import os
import matplotlib.pyplot as plt

## Entorno del Proyecto

El entorno representa una cinta transportadora con productos:  
(estados)
- 0 → producto bueno
- 1 → producto defectuoso

Acciones del agente:
- 0 = dejar pasar
- 1 = retirar

Recompensas:
- +10 si toma la decisión correcta
- -5 si se equivoca


In [2]:
class CintaTransportadoraEnv:

    def __init__(self, prob_malo=0.3, semilla=None):
        self.prob_malo = prob_malo
        self.rng = random.Random(semilla)
        self.producto_actual = None

    def reiniciar(self):
        
        self.producto_actual = 1 if self.rng.random() < self.prob_malo else 0
        return self.producto_actual

    def paso(self, accion):
        
     
   
        prod = self.producto_actual
        if accion == 1:  # retirar
            recompensa = 10.0 if prod == 1 else -5.0
        else:            # dejar pasar
            recompensa = 5.0 if prod == 0 else -10.0

        self.producto_actual = 1 if self.rng.random() < self.prob_malo else 0
        terminado = False
        info = {"producto": prod}
        return self.producto_actual, recompensa, terminado, info


## Algoritmo Q-Learning

El agente aprende una tabla Q con valores para cada acción.  
Actualiza la tabla con la fórmula:

Q(s,a) = Q(s,a) + α [ r + γ max(Q(s’, a’)) − Q(s,a) ]

Con esto aprende qué hacer en cada estado.


In [3]:
def entrenamiento(episodios, alpha, gamma,epsilon, decay_epsilon, min_epsilon,prob_malo, semilla, guardar_ruta):
    env = CintaTransportadoraEnv(prob_malo=prob_malo,semilla=semilla)
    n_estados=2
    n_acciones=2
    
    #iniciamos en ceros
    Q = np.zeros((n_estados,n_acciones),dtype=np.float64)
    historial_recompensas=[]
    eps= epsilon
    
    #iteramos en los episodios
    for ep in range (episodios):
        estado = env.reiniciar()
        
        #aqui iniciamos la politica epsilon grady
        if np.random.rand() < eps:
            accion = np.random.randint(n_acciones)
        else:
            accion = int(np.argmax(Q[estado]))
        
        siguiente_estado, recompensa, terminado, info = env.paso(accion)
        
        mejor_siguiente = np.max(Q[siguiente_estado])
        td_objetivo = recompensa + gamma * mejor_siguiente
        td_error = td_objetivo - Q[estado, accion]
        Q[estado, accion] += alpha * td_error

        historial_recompensas.append(recompensa)
        eps = max(min_epsilon, eps * decay_epsilon)
        
        if (ep + 1) % 2000 == 0:
            prom_reciente = np.mean(historial_recompensas[-5000:])
            print(f"Episodio {ep+1}/{episodios} | recompensa_promedio_ult_{2000}: {prom_reciente:.2f} | eps: {eps:.4f}")

    np.save(guardar_ruta, Q)
    print(f"Entrenamiento finalizado. Q-tabla guardada en: {guardar_ruta}")
    return Q, historial_recompensas
    

## Entrenamiento del Agente

El agente juega miles de veces.  
Usa "epsilon-greedy": explora acciones nuevas y luego aprovecha lo que ya aprendió.  
Con el tiempo comete cada vez menos errores.


In [ ]:
Q, recompensas = entrenamiento(episodios=20000, alpha=0.2, gamma=0.99,epsilon=0.4, decay_epsilon=0.9998, min_epsilon=0.01,prob_malo=0.3, semilla=123, guardar_ruta='q_tabla.npy')


Episodio 2000/20000 | recompensa_promedio_ult_2000: 4.21 | eps: 0.2681
Episodio 4000/20000 | recompensa_promedio_ult_2000: 4.58 | eps: 0.1797
Episodio 6000/20000 | recompensa_promedio_ult_2000: 5.19 | eps: 0.1205
Episodio 8000/20000 | recompensa_promedio_ult_2000: 5.69 | eps: 0.0807
Episodio 10000/20000 | recompensa_promedio_ult_2000: 5.95 | eps: 0.0541
Episodio 12000/20000 | recompensa_promedio_ult_2000: 6.12 | eps: 0.0363
Episodio 14000/20000 | recompensa_promedio_ult_2000: 6.16 | eps: 0.0243
Episodio 16000/20000 | recompensa_promedio_ult_2000: 6.33 | eps: 0.0163
Episodio 18000/20000 | recompensa_promedio_ult_2000: 6.38 | eps: 0.0109
Episodio 20000/20000 | recompensa_promedio_ult_2000: 6.44 | eps: 0.0100
Entrenamiento finalizado. Q-tabla guardada en: q_tabla.npy


## simulacion con pygame  

en esta seccion lo que hacemos es crear un funcion para poder simularlo  


In [5]:
import pygame

def simular_pygame(q_tabla_ruta='q_tabla.npy', prob_malo=0.3):
    if not os.path.exists(q_tabla_ruta):
        print(f"Q-tabla no encontrada en {q_tabla_ruta}. Entrena primero.")
        return
    Q = np.load(q_tabla_ruta)

    pygame.init()
    ANCHO, ALTO = 640, 240
    pantalla = pygame.display.set_mode((ANCHO, ALTO))
    pygame.display.set_caption('Cinta Transportadora - Simulación Q-Learning')
    reloj = pygame.time.Clock()
    FPS = 60

    BLANCO = (255,255,255)
    CINTA = (120,120,120)
    BUENO = (50,180,50)
    MALO = (200,60,60)
    RETIRADOR = (70,130,180)

    x_producto = 40
    y_producto = ALTO//2
    velocidad = 2
    rng = random.Random(123)
    estado_producto = 1 if rng.random() < prob_malo else 0

    corriendo = True
    while corriendo:
        for evento in pygame.event.get():
            if evento.type == pygame.QUIT:
                corriendo = False

        pantalla.fill(BLANCO)
        pygame.draw.rect(pantalla, CINTA, (0, ALTO//2 - 20, ANCHO, 40))
        estado = estado_producto
        accion = int(np.argmax(Q[estado]))
        x_retirador = ANCHO - 200
        if accion == 1:
            pygame.draw.rect(pantalla, RETIRADOR, (x_retirador, ALTO//2 - 60, 40, 120))
        else:
            pygame.draw.rect(pantalla, (180,180,180), (x_retirador, ALTO//2 - 60, 40, 120))

        color = BUENO if estado_producto == 0 else MALO
        pygame.draw.circle(pantalla, color, (int(x_producto), y_producto), 16)
        x_producto += velocidad
        if x_producto >= x_retirador + 10:
            if accion == 1:
                y_producto -= 4
            else:
                y_producto = ALTO//2
        if x_producto > ANCHO + 50 or y_producto < -50:
            x_producto = 40
            y_producto = ALTO//2
            estado_producto = 1 if rng.random() < prob_malo else 0

        fuente = pygame.font.SysFont(None, 20)
        texto = fuente.render('Política Q-table: acción=argmax(Q[estado])  |  acción 0=dejar 1=retirar', True, (0,0,0))
        pantalla.blit(texto, (10,10))
        pygame.display.flip()
        reloj.tick(FPS)

    pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/guxs/anaconda3/envs/iapro/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


## ejecucion de la simulacion  
al ejecutar la simulacion lo que estamos haciendo es pasar la tabla Q que tenemos almacenada

In [6]:
simular_pygame('q_tabla.npy')
